### Mask out regions of the microscope timelpase. Here, PDMS microchannels.

This notebook guides through the process of semi-manually segmenting the microchannels from the microscope transmission channel. The obtained mask sparsifies the timelapse data significantly and is needed for computing A* distances through the microstructure. First load the transmission channel into napari, the image viewer gui. Perform prewitt edge detection, gaussian denoising, binarize using otsu, run binary closing. Then comes the manual part where the channels are filled using the bucket tool. Once the has been done, use flood to clean the PDMS structure channels.

In [1]:
import napari
from tifffile import imread

import numpy as np
import pandas as pd

from skimage import filters
from skimage import morphology
from skimage.filters import threshold_otsu
from skimage.segmentation import flood

import matplotlib.pyplot as plt

In [2]:
def load_transmission_channel(basename, postfix='_Transmission_compr.deflate'):
    """
    Load the tif transmission channel of a timelapse recording. Will try to get
    the file {data_path}/{basename}{postfix}.tif.
    
    Arguments
    ---------
        basename: str
            The basename of the transmission channel/ timelapse to load. This
            prefix is consistent for all types of data corresponding to this
            microscope timelapse.
        postfix: str
            Load a file that ends with postfix (excluding file extension), by
            default `_Transmission_compr.deflate`.
    Returns
    -------
        transm_chnl: np.array of shape [HxW], dtype uint16
            The loaded transmission channel.
    """
    transm_chnl_fname = f'{data_path}/{basename}{postfix}.tif'
    print(f'Loading transmission channel for {basename}: {transm_chnl_fname}')
    transm_chnl = imread(transm_chnl_fname)
    if transm_chnl.ndim == 3:
        print(f'Transmission channel had temporal dim [{transm_chnl.shape[0]}].'
              f' Slicing to t=0')
        transm_chnl = transm_chnl[0]
    return transm_chnl

In [3]:
def transmission_channel2gui(transm_chnl):
    """
    Load the transmission channel numpy array into the image viewer. Will get 
    the name `0_original`.
    
    Arguments
    ---------
        transm_chnl: np.array of shape [HxW], dtype uint16
            The transmission channel to be loaded into the GUI.
    """
    viewer.add_image(transm_chnl, name='0_original', blending='additive', 
                     opacity=.7, gamma=.72)

In [4]:
def segment_microchannels(transm_chnl, gaussion_sigma=1, bin_closing_dim=4):
    """
    Segment the microchannels using filters. Performs prewitt edge detection,
    gaussian denoising, binarize using otsu, and finally binary closing. Loads
    the in-between results into the image viewer as well. Returns the initial
    mask segmentation.
    
    Arguments
    ---------
        transm_chnl: np.array of shape [HxW], dtype uint16
            The transmission channel to segment.
        gaussion_sigma: int
            The stregth of gaussion smoothing post edge detection. Defaults to
            1.
        binary_closing_dim: int
            The extend of binary clossing. Increase if edges have frequent gaps.
            Decrease if edge are frequently merged together. Defaults to 4.
    Returns
    -------
        initial_mask: np.array of shape [HxW], dtype bool
            The inital mask obtained from filtering.
    """
    print('Detecting edges using prewitt filter', end='...')
    # perform edge detection
    prewitt = filters.prewitt(transm_chnl)
    viewer.add_image(prewitt, visible=False, name='1_prewitt_edge_detection')
    
    print('smoothing edges using gaussion filter', end='...')
    # perform guassian smoothing on edge filtered image
    prewitt_gaussian = filters.gaussian(prewitt, sigma=gaussion_sigma)
    viewer.add_image(prewitt_gaussian, name='2_smoothed_edges', visible=False)
    
    print('threshold using otsu', end='...')
    # binarize image
    prewitt_bin = prewitt > threshold_otsu(prewitt_gaussian)
    viewer.add_image(prewitt_bin, name='3_otsu_threshold', visible=False)
    
    print('binary closing', end='...')
    # close edges as good as possible
    square = morphology.square(bin_closing_dim)
    initial_mask = morphology.binary_closing(prewitt_bin, square)
    viewer.add_labels(initial_mask, name='4_bin_closing_initial_mask')
    
    print('Done.')
    return initial_mask
    

In [5]:
def add_floodpoint_layer():
    """
    Add a points-layer to the viewer. Manually set the flood point after adding
    this.
    """
    viewer.add_points(name='flood_point')

In [6]:
def get_floodpoint():
    """
    Get the flood point from the previously added points-layer.
    
    Returns
    -------
        floodpoint: (int, int)
            XY coordinate from which to flood.
    """
    floodpoint = viewer.layers['flood_point'].data
    return tuple(floodpoint[0].astype(int))

In [7]:
def flood_initial_mask(floodpoint):
    """
    Flood the manually cleaned segmentation from a passed point. This ensures
    that everything is connected.
    
    Arguments
    ---------
        floodpoint: (int, int)
            XY coordinate from which to flood.
    """
    filled_mask = viewer.layers['4_bin_closing_initial_mask'].data    
    filled_mask_flooded = flood(filled_mask, floodpoint)
    viewer.add_labels(filled_mask_flooded, name='5_filled_mask_flooded')

In [22]:
def save_final_mask(basename, postfix='_Transmission_compr.deflate_mask1'):
    """
    Save the final mask from the image viewer by its name:
    `5_filled_mask_flooded`. Filename will be
    {data_path}/{basename}{postfix}.npy.
    
    Arguments
    ---------
        basename: str
            The basename of the transmission channel/ timelapse to save. This
            prefix is consistent for all types of data corresponding to this
            microscope timelapse.
        postfix: str
            Save a file that ends with postfix (excluding file extension), by
            default `_Transmission_compr.deflate_mask1`.
    """
    final_mask = viewer.layers['5_filled_mask_flooded'].data.astype(bool)
    print(final_mask)
    fname = f'{data_path}/{basename}{postfix}.npy'
    print(f'Saving mask: {fname}')
    np.save(fname, final_mask)

### Run below for segmentation

In [9]:
# get the load_mask function
%run 01_process_training_timelapses.ipynb

Loaded functions `load_timelapse`, `load_mask` successfully.


In [10]:
# create the viewer for segmentation
viewer = napari.Viewer()

In [11]:
# set the dataset names
# data_path = '/run/media/loaloa/lbbSSD/training_data/'
data_path = '/run/media/loaloa/lbbSSD/test_axtrack_inference/'
# tlapse_names = ['D00_G001', 'D04_G004', 'D19_G035']
tlapse_names = ['timelapse42',]


# segment dataset at index 
index = 0
basename = tlapse_names[index]

In [13]:
# load the transmission tif file and put in in the gui
transm_chnl = load_transmission_channel(basename)
transmission_channel2gui(transm_chnl)

Loading transmission channel for timelapse42: /run/media/loaloa/lbbSSD/test_axtrack_inference//timelapse42_Transmission_compr.deflate.tif
Transmission channel had temporal dim [1]. Slicing to t=0


In [14]:
# do a sequence of classic image preprocessing to segment the micro channels
gaussion_sigma = 1    # for smoothing after edge detection
bin_closing_dim = 4     # how strongly to close binarized edges
inital_mask = segment_microchannels(transm_chnl, gaussion_sigma, bin_closing_dim)

Detecting edges using prewitt filter...smoothing edges using gaussion filter...threshold using otsu...binary closing...Done.


### Now use the bucket tool to fill the microchannels inside. Close edges where neseccary.


In [15]:
# next set a point to fill from 
add_floodpoint_layer()

In [16]:
# fill from set point to create one all-connected channel system
fp = get_floodpoint()
flood_initial_mask(fp)

In [23]:
# save the final mask
save_final_mask(basename)

[[False False False ... False False False]
 [False False False ... False False False]
 [False False False ... False False False]
 ...
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]
 [ True  True  True ... False False False]]
Saving mask: /run/media/loaloa/lbbSSD/test_axtrack_inference//timelapse42_Transmission_compr.deflate_mask1.npy
